In [1]:
import numpy as np
import pandas as pd

import torch 
import torch.nn as nn
import optuna
from datetime import datetime, timedelta
from src.split import train_test_split

ModuleNotFoundError: No module named 'src'

# Data Preprocessing

In [35]:
# import feature data
feature = pd.read_csv('/Users/mac/Desktop/PycharmProjects/TAADL/DATA/features.csv', index_col='Date')
feature.index = pd.to_datetime(feature.index)
tr_x, tr_y, val_x, val_y, te_data = train_test_split(feature)

# Train-Val-Test Split

In [177]:
def train_test_split(data:pd.DataFrame, valid_ratio:float=0.2, train_ratio:float=0.6):
    # we will use 60% of the data as train+val set. this data will be used to tune DL model.
    n = data.shape[0]
    tr_idx  = int(n * train_ratio)
    val_idx = int(tr_idx * (1-valid_ratio))

    # split the dataset according to tr index
    tr_data = data.iloc[:tr_idx]
    te_data = data.iloc[tr_idx:]

    # Now we create a target for training a DL model. Target will be a future 130-days return of each ETF products.
    # To make a target we look 130 days ahead of the training set. This will be done by shifting the view by 130 days.
    target_list = ['BNDX_ret130','BND_ret130','VGK_ret130','VNQI_ret130',
                    'VNQ_ret130','VTI_ret130','VWOB_ret130','VWO_ret130']
    target = feature.iloc[130:tr_idx+130][target_list]

    # add prefix on columns and reset the
    target = target.add_prefix('targ_')
    target.index = tr_data.index

    assert tr_data.shape[0] == target.shape[0], 'feature shape and target shape is not same.'
    mu, std = tr_data.mean(axis=0), tr_data.std(axis=0)
    tr_data = tr_data.sub(mu).div(std) # normalize the data along rows

    trn_features, trn_label = tr_data.iloc[:val_idx-130], target.iloc[:val_idx-130] # cut last few days of data to prevent data leakage.
    val_features, val_label = tr_data.iloc[val_idx:], target.iloc[val_idx:]

    print(trn_features.shape, trn_label.shape)
    print(val_features.shape, val_label.shape)
    return trn_features, trn_label, val_features, val_label, te_data

In [178]:
tr_x, tr_y, val_x, val_y, te_data = train_test_split(feature)

(1702, 46) (1702, 8)
(458, 46) (458, 8)


# Network Training Scheme

첫번째 파트는 향후 6개월의 예상 수익률을 예측하는 것을 output 으로 목표로 하고 training, validation set 을 이용해 MLP 를 학습시켜 주는 것입니다.
여기서는 Cross-validation 이 아닌 Walk-forward method 를 이용해서 데이터가 점진적으로 늘어나는 세팅을 사용하였고, batch 의 약 30%는 validation set 으로 loss 를 계산했습니다. 

hyperparameter 중 몇 개는 논문 원문에서 가져왔고, model update epoch 는 임의로 설정했습니다. (여기서는 `n=30`) 그 외의 hyperparameter 는 `optuna`를 이용하여 validation loss 가 낮아지는 것을 objective 로 두고 search 했습니다.

[I 2022-02-26 17:48:07,705] A new study created in memory with name: no-name-75ad5053-a23a-42ce-8af0-a15d0d13704d
[I 2022-02-26 17:48:10,512] Trial 0 finished with value: 0.06572332978248596 and parameters: {'learning rate': 0.004931038571464121, 'weight decay': 1.3934813638959082e-05, 'epsilon': 0.060871899363027296}. Best is trial 0 with value: 0.06572332978248596.


Average Training loss : 0.17898411806672812, Test Loss: 0.06572332978248596


[I 2022-02-26 17:48:12,600] Trial 1 finished with value: 0.2250470221042633 and parameters: {'learning rate': 0.003330843820651318, 'weight decay': 0.006494655367292259, 'epsilon': 0.48000629719415866}. Best is trial 0 with value: 0.06572332978248596.


Average Training loss : 0.7733308918774128, Test Loss: 0.2250470221042633


[I 2022-02-26 17:48:14,722] Trial 2 finished with value: 0.41273871064186096 and parameters: {'learning rate': 0.003179561548618473, 'weight decay': 3.1712238082907e-06, 'epsilon': 0.7079124731989297}. Best is trial 0 with value: 0.06572332978248596.


...

Average Training loss : 0.06700747013092041, Test Loss: 0.009159051813185215


[I 2022-02-26 17:51:14,279] Trial 98 finished with value: 0.00957111269235611 and parameters: {'learning rate': 0.004297168844310529, 'weight decay': 0.02167961106894991, 'epsilon': 0.006967847036147487}. Best is trial 97 with value: 0.009159051813185215.


Average Training loss : 0.03962856903672218, Test Loss: 0.00957111269235611


[I 2022-02-26 17:51:16,073] Trial 99 finished with value: 0.009620252065360546 and parameters: {'learning rate': 0.004867305387030955, 'weight decay': 0.022915270062657925, 'epsilon': 0.008829338157094191}. Best is trial 97 with value: 0.009159051813185215.


Average Training loss : 0.03165750531479716, Test Loss: 0.009620252065360546


In [196]:
default_params = study.best_trial.params

In [204]:
default_params

{'learning rate': 0.00417260629089144,
 'weight decay': 0.022776370530345223,
 'epsilon': 0.04453309165648957}

In [205]:
# call the network for training
model = MLP()
model.weight_init()

# set up the loss function and optimizer
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=default_params['learning rate'], weight_decay=default_params['weight decay'], eps=default_params['epsilon'])

t1 = pd.Series(feature[130:tr_x.shape[0] +130].index, index=tr_x.index) # look ahead of 130 days
cv = PurgedKFold(n_splits=5, t1=t1, pctEmbargo=0.05) 

train_losses = []
for fold_idx, (tr, val) in enumerate(cv.split(tr_x)): # 5-fold Purged Cross Validation
    # assign training set and validation set of CV scheme
    x_train, y_train = torch.Tensor(tr_x.values[tr]), torch.Tensor(tr_y.values[tr])
    x_valid, y_valid = torch.Tensor(tr_x.values[val]), torch.Tensor(tr_y.values[val])
    
    # for each epoch train and update the network
    for epoch in range(100):
        optimizer.zero_grad()
        
        # get prediction and its loss with gradients
        output = model(x_train)
        loss = loss_fn(output, y_train)
        loss.backward()

        # backpropagation
        optimizer.step()
        
    # validate model
    output = model(x_valid)
    valid_loss = loss_fn(output, y_valid)  
    train_losses.append(valid_loss.detach().item())  

# after training and CV
x_test, y_test = torch.Tensor(val_x.values), torch.Tensor(val_y.values)
output = model(x_test).detach()
test_loss = loss_fn(output, y_test)
print(f'Average Training loss : {np.mean(train_losses)}, Test Loss: {test_loss}')

Average Training loss : 0.11654255986213684, Test Loss: 0.01178944855928421


In [237]:
torch.save({'net_state_dict':model.state_dict(), 'optimizer_dict':optimizer.state_dict()}, '/Users/mac/Desktop/PycharmProjects/TAADL/network/params.json')